In [ ]:
# parameter cell
DEVICES = "1,2"
FROM = "2019-01-01"
TO = "2020-01-01"
COUNTRY = "Vietnam"
ORDER = 1  # will be pass to param order in biLouvian call

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = DEVICES

import re
import cupy as cp
import pandas as pd
from tqdm import tqdm
import cudf
import networkx as nx
from cuml.preprocessing import LabelEncoder

tqdm.pandas()

# Import raw data and filter by the Paramteres

In [ ]:
ds = cudf.read_parquet("../fulldata/kiva_activity_2023-08-28T11-09-39.parquet")
ds["sector_name"] = ds["sector_name"].astype("category")
ds["geocode_country_name"] = ds["geocode_country_name"].astype("category")
ds["activity_name"] = ds["activity_name"].astype("category")

In [ ]:
ds.dropna(axis=0, how="all", inplace=True)
print(len(ds))
ds.tail(2)

In [ ]:
if COUNTRY.lower() != "all":
    ds = ds[ds["geocode_country_name"] == COUNTRY]
len(ds)

Try to limit the timeline, because I am not yet comfortable to work with large data

In [ ]:
ds = ds[(ds.fundraisingDate >= FROM) & (ds.fundraisingDate < TO)]
"the number of Loans (might duplicated) under investigation is", len(ds)

# Basic process

In [ ]:
ds.rename(columns={"id": "project_id", "name": "project_name"}, inplace=True)

# Preprocessing

## Remove duplicated Projects

There are Projects which have a same `project_id` but different `fundedAmount`
It might because the query time is different
Here, only keep records which have the highest `fundedAmount`

In [ ]:
temp = ds.groupby("project_id", group_keys=False)[["loanFundraisingInfo_fundedAmount"]].idxmax()
iloc = temp["loanFundraisingInfo_fundedAmount"].values  # NOTE: just iloc, not loc
ds = ds.iloc[iloc]
del iloc
del temp

In [ ]:
assert 0 == len(ds[ds.duplicated(subset=["project_id"], keep=False)].sort_values(by=["project_id"]))  # no duplicated

In [ ]:
"the number of Projects (no duplicated) under investigation is", len(ds)

## create `Lender-Project-Sector` df

In [ ]:
ads = ds.explode("lendingActions_values")
len(ads)

In [ ]:
# drop some loans that has no lender
ads.dropna(subset=["lendingActions_values"], inplace=True)

In [ ]:
# dict processing cant be done in cuDF, so convert to pandas
adf = ads.to_pandas()

In [ ]:
adf["lender_id"] = adf.progress_apply(lambda x: x["lendingActions_values"]["lender"]["id"], axis=1).astype(int)
adf["lender_name"] = adf.progress_apply(lambda x: x["lendingActions_values"]["lender"]["name"], axis=1)
adf["lender_publicId"] = adf.progress_apply(lambda x: x["lendingActions_values"]["lender"]["publicId"], axis=1)
adf["loan_shareAmount"] = adf.progress_apply(lambda x: x["lendingActions_values"]["shareAmount"], axis=1).astype(float)
adf["loan_date"] = pd.to_datetime(
    adf.progress_apply(lambda x: x["lendingActions_values"]["latestSharePurchaseDate"], axis=1)
)

In [ ]:
# cuDF do not work with timezone yet
adf["loan_date"] = adf["loan_date"].dt.tz_localize(None)

In [ ]:
adf.drop(["lendingActions_values"], axis=1, inplace=True)

In [ ]:
ads = cudf.from_pandas(adf)
del adf

In [ ]:
assert 0 == ads.duplicated(subset=["project_id", "lender_id", "loan_shareAmount", "loan_date"]).sum()

In [ ]:
ads["loan_id"] = cp.arange(len(ads["lender_id"])) + 1

In [ ]:
ads.duplicated().sum()  # NOTE: only work with small dataset

In [ ]:
ads.drop_duplicates(inplace=True)  # NOTE: only work with small dataset

## Remove anonymous Lenders

In [ ]:
print("anynomous Lenders")
anons = ads.lender_publicId.str.startswith("anon")
ads[anons][["lender_id", "lender_name", "lender_publicId"]].drop_duplicates()

In [ ]:
# drop those anon
ads = ads[~anons]

In [ ]:
ads.drop_duplicates(inplace=True)

In [ ]:
ads.reset_index(inplace=True, drop=True)

# Stats

In [ ]:
"number of Loans", ads.project_id.unique().count()

In [ ]:
"number of Lenders", ads.lender_id.unique().count()

# Contruct `Lender-Sector` bipartite graph

## Create edge list of the graph

In [ ]:
ads.info()

In [ ]:
LS = ads.groupby(["lender_id", "lender_publicId", "sector_id", "sector_name"]).agg(
    {"sector_name": "first", "loan_shareAmount": "sum", "loan_date": "count"}
)
LS.reset_index(inplace=True)
LS.rename(columns={"loan_shareAmount": "loan_amount", "loan_date": "loan_count"}, inplace=True)
LS = LS[LS["loan_amount"] > 0]
LS

## Convert to the format that `biLouvian` can understand

In [ ]:
le1 = LabelEncoder()
LS["V1"] = le1.fit_transform(LS["sector_name"])
le2 = LabelEncoder()
LS["V2"] = le2.fit_transform(LS["lender_id"]) + LS["V1"].max() + 1
LS.tail(3)

In [ ]:
# total number of vertex
vertex_count = LS["lender_id"].nunique() + LS["sector_name"].nunique()
assert vertex_count == LS["V1"].nunique() + LS["V2"].nunique()
print(vertex_count)

In [ ]:
dictionary1 = LS[["V1", "sector_name"]].drop_duplicates()
dictionary1.rename(columns={"sector_name": "name", "V1": "id"}, inplace=True)
dictionary1["name"] = dictionary1["name"].astype(str)
dictionary1.sort_values(by=["id"]).tail(3)

In [ ]:
dictionary2 = LS[["V2", "lender_publicId"]].drop_duplicates()
dictionary2.rename(columns={"lender_publicId": "name", "V2": "id"}, inplace=True)
dictionary2.head(1)

In [ ]:
# concat two dictionaries
dictionary = cudf.concat([dictionary1, dictionary2])
print(len(dictionary))
assert len(dictionary) == vertex_count
dictionary.tail()

In [ ]:
prefix = f"checkpoints/LS_bipartite_{COUNTRY}_from{FROM}_to{TO}_order{ORDER}"  # the text "bipartite" is mandatory
filename = f"{prefix}.csv"
dictfile = f"{prefix}_Dictionary.txt"  # "_Dictionary.txt" is mandatory
LS[["V1", "V2", "loan_amount"]].to_csv(filename, sep="\t", header=False, index=False)  # "\t" is mandatory
dictionary[["id", "name"]].to_csv(dictfile, sep="\t", header=False, index=False)

# Run `biLouvian`

Note: Have to build the `biLouvian` binary first. Refer to `doc/BUILD_BILOUVIAN.md`

In [ ]:
!../extra/biLouvian -i $filename -order $ORDER